In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from functools import reduce


In [2]:
assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/assessments.csv')
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/courses.csv')
regs = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/studentRegistration.csv')
st_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/studentAssessment.csv')


# Задание 1

<font size = 4> Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене). </font>

Нам не хватает данных о типе ассессмента, поэтому мы вытакскиваем его из таблицы с информацией об всех формах оценивания студентов.

In [3]:
st_assessment = pd.merge(st_assessment, 
                        assessment[['id_assessment', 'assessment_type','code_module','code_presentation']], 
                        on = 'id_assessment')

Делаем предварительную фильтрацию данных: оставляем только тех студентов, которые набрали 40 и больше баллов за экзамен. Также считаем, как много таких студентов было в каждом семестре для каждого предмета.

In [4]:
st_assessment.query('score >= 40 and assessment_type == "Exam"')\
             .groupby(['code_module', 'code_presentation'])\
             .agg({'id_student': 'count'})

id_student
code_module code_presentation            
CCC         2014B                     664
            2014J                    1019
DDD         2013B                     504
            2013J                     878
            2014B                     485
            2014J                     842

 Видно, что в семестре 2014B и семестре 2014J были студенты, которые одновременно успешно сдали экзамен по курсу CCC и DDD. По условию задания нам нужно назвать количество студентов, которые сдали успешно закончили **только один курс**. Сохраним в отдельных таблицах студентов, которые успешно закончили курс CCC и BBB.

In [5]:
ccc_exam = st_assessment.query('score >= 40 and assessment_type == "Exam" and code_module == "CCC"')
ddd_exam = st_assessment.query('score >= 40 and assessment_type == "Exam" and code_module == "DDD"')

Деалем *outer join* двух новых датафреймов, чтобы найти множество студентов, которые успешно сдали только один экзамен.

In [6]:
merged = pd.merge(ddd_exam[['id_student','score', 'code_module', 'code_presentation', 'date_submitted','id_assessment']], 
         ccc_exam[['id_student','score', 'code_module','code_presentation', 'date_submitted','id_assessment']],
         how = 'outer',
         on = 'id_student')

На всякий случай перепроверим наличие дубликатов в столбце **id_student**.

In [7]:
id_duplicates = list(merged[merged.duplicated(subset='id_student')].iloc[:,0])
merged.query('id_student in @id_duplicates').iloc[:,:6]

,id_student,score_x,code_module_x,code_presentation_x,date_submitted_x,id_assessment_x
284,540758,40.0,DDD,2013B,229.0,25340.0
285,540758,60.0,DDD,2014B,234.0,25361.0
753,593903,49.0,DDD,2013J,241.0,25354.0
754,593903,44.0,DDD,2014J,243.0,25368.0
860,605118,42.0,DDD,2013J,237.0,25354.0
861,605118,64.0,DDD,2014J,243.0,25368.0


У нас есть три студента, которые зачем-то пересдавали экзамен через год при том, что они набрали необходимое количество баллов при первой сдаче экзамена. Чтобы получить ответ, удалим дубликаты из объединенного датафрейма.

In [8]:
ans = merged.drop_duplicates(subset='id_student').shape[0]
print(f'{ans} студентов успешно сдали только один курс.')

4097 студентов успешно сдали только один курс.


# Задание 2

<font size = 4> Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью*.</font>

*завершаемость = кол-во успешных экзаменов / кол-во всех попыток сдать экзамен*

In [9]:
hard_exams = st_assessment.query('assessment_type == "Exam"')

In [10]:
hard_exams['status'] = st_assessment['score'].apply(lambda x: 'pass' if x >= 40 else 'no_pass')

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
hard_exams.groupby(['code_module','id_assessment'])['status']\
          .value_counts(normalize=True)\
          .to_frame().unstack()\
          .sort_values(by=('status','pass'))

status          
status                      no_pass      pass
code_module id_assessment                    
DDD         25340          0.162791  0.837209
CCC         24299          0.127568  0.872432
DDD         25368          0.113684  0.886316
CCC         24290          0.111111  0.888889
DDD         25354          0.092975  0.907025
            25361          0.074427  0.925573

Самая высокая заверашемость у курса DDD 25361 — 92.5% сдали экзамен. Самая низкая завершаемость у курса CCC 24299 — 87.2%. id_assessment 25340



# Задание 3

<font size = 4> По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом). </font>

In [12]:
time_exams = st_assessment.query('assessment_type == "Exam" and score >= 40')

In [13]:
time_exams.groupby('code_module').agg({'date_submitted':['mean','median']})

date_submitted       
                      mean median
code_module                      
CCC             239.345217    244
DDD             237.976006    237

В среднем студенты курса CCC успешно сдают экзамен на 239 день с момента начала семестра, тогда как курс DDD студенты завершают на 237 день. При этом надо учитывать, что время прохождения курса может отличаться в зависимости от семестра, в который он проводится. Эту разницу можно оценить ниже.

In [14]:
time_exams.groupby(['code_module','code_presentation']).agg({'date_submitted':['mean','median']})

date_submitted       
                                        mean median
code_module code_presentation                      
CCC         2014B                 231.581325    230
            2014J                 244.404318    244
DDD         2013B                 230.164683    230
            2013J                 239.509112    241
            2014B                 234.936082    234
            2014J                 242.804038    243

# Задание 4

<font size = 4> Выяви самые популярные курсы (ТОП-3) по количеству регистраций на них. А также курсы с самым большим оттоком (ТОП-3). </font>

Предварительно проверим, есть ли пропуски в полях date_registration и date_unregistration. Мы видим, что очень много пропусков в date_unregistration. Это говорит о том, что NaN указывался, если студент не отписывался от курса.

In [15]:
regs.isna().sum()

code_module                0
code_presentation          0
id_student                 0
date_registration         45
date_unregistration    22521
dtype: int64

Посмотрим внимтельнее на пропуски в date_registration. Мы видим, что в таблице не зафиксирован день записи, но при этом есть день отмены регистрации. Скорее всего это какая-то ошибка, которая появилась на этапе составление таблицы.

In [16]:
regs.query('date_registration == "NaN"')

,code_module,code_presentation,id_student,date_registration,date_unregistration
2344,BBB,2013B,630346,NaN,NaN
2538,BBB,2013J,57369,NaN,-1.0
2759,BBB,2013J,342678,NaN,-33.0
5356,BBB,2014B,582496,NaN,-126.0
5490,BBB,2014B,607646,NaN,-38.0
5573,BBB,2014B,614270,NaN,-142.0
6295,BBB,2014B,2409808,NaN,-109.0
6305,BBB,2014B,2439442,NaN,-149.0
8307,BBB,2014J,694001,NaN,-36.0
8975,CCC,2014B,394791,NaN,-61.0


Заолним столбец единицами, чтобы эти строки учитывались при группировке.

In [17]:
regs['date_registration'].fillna(value=1,inplace=True)

In [18]:
regs.groupby('code_module').agg({'date_registration':'count', 'date_unregistration':'count'})\
                           .sort_values(by=['date_registration','date_unregistration'],ascending=False)\
                           .iloc[:3,:]

,date_registration,date_unregistration
code_module,,
BBB,7909,2377
FFF,7762,2380
DDD,6272,2235


Самые популярные курсы по количеству регистраций являются же и курсами с самым больши оттоком.

# Задание 5

<font size = 4> Напиши функцию на python, позволяющую строить когортный (семестровый) анализ. В период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов. </font>

In [19]:
assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/assessments.csv')
st_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/studentAssessment.csv')


In [20]:
def cohort_elearning(st_assessment, assessment):
    
    '''
    Первый аргумент функции — это датафрейм, содеражащий результаты тестов студентов. 
    Во второй аргумент передается датафрейм с информацией об оценках в тесте.

    Пояснение к работе функции:

    1) Сначала мы объединяем два датасета, чтобы итоговая таблица содержала инфомрацию о типе ассесмента;
    2) После фильтрации оставялем информацию о студентах, которые успешно сдали экзамен;
    3) Создаем сводную таблицу success, в которой будет указан процент завершенных курсов;
    4) Еще одна сводная таблица time_to_pass, где хранится информация о среднем времени, которое требуется для завершения курса;
    5) Выводим итоговую сводную таблицу.
    
    '''
    
    st_assessment = pd.merge(st_assessment, 
                        assessment[['id_assessment', 'assessment_type','code_module','code_presentation']], 
                        on = 'id_assessment')
    
    time_exams = st_assessment.query('assessment_type == "Exam" and score >= 40')
    
    success = hard_exams.groupby('code_presentation')['status'].value_counts(normalize=True)\
                                                 .to_frame()\
                                                 .rename(columns={'status': 'success'})\
                                                 .reset_index(level=['status','code_presentation'])\
                                                 .query('status == "pass"').iloc[:,[0,2]]
    
    time_to_pass = hard_exams.groupby('code_presentation',as_index=False).agg({'date_submitted':'mean'})
    
    return pd.merge(success, time_to_pass, on='code_presentation')
    
    

In [21]:
cohort_elearning(st_assessment, assessment)

,code_presentation,success,date_submitted
0,2013B,0.837209,230.179402
1,2013J,0.907025,239.505165
2,2014B,0.904013,232.988985
3,2014J,0.878659,243.674221


Самая низкая завершаемость курсов была в семестре **2013B**. В среднем курсы сдавали дольше всего в семестре **2014J**.

# Задание 6

<font size = 4> Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: **R - среднее время сдачи одного экзамена**, **F - завершаемость курсов**, **M - среднее количество баллов**, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 0.1 до 0.5, monetary от 55 до 72 баллов. </font>

In [22]:
assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/assessments.csv')
st_assessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-s-kolesov/studentAssessment.csv')

st_assessment = pd.merge(st_assessment, 
                        assessment[['id_assessment', 'assessment_type','code_module','code_presentation']], 
                        on = 'id_assessment')

Начнем с расчета **R- среднее время сдачи одного экзамена**. Мы загрузили данные, теперь оставим только строки со студентами, которые сдавали экзамен.

In [23]:
st_assessment = st_assessment.query('assessment_type == "Exam"')

Сгруппируем данные по id студента и посчитаем, сколько времени в среднем потратил студент для сдачи экзмена, при этом мы не учитываем, сдал ли он его успешно или нет.

In [24]:
date_submitted_mean = st_assessment.groupby('id_student', as_index = False).agg({'date_submitted':'mean'})

Мы создадим категориальные переменные на основе квантилей: 

0 - 0.3 соответсвует студентами, которые затратили времени в среднем меньше, чем 2/3 других студентов; 
0.3 - 0.7 студенты-среднячки, потратили не слишком много времени, но при этом не слишком мало;
0.7 - 1 студенты, потратившие времени больше всех остальных на сдачу экзамена

In [25]:
bins = [0, 
        date_submitted_mean['date_submitted'].quantile(q=0.3),
        date_submitted_mean['date_submitted'].quantile(q=0.7),
        date_submitted_mean['date_submitted'].max()]

labels = [1, 2, 3]

date_submitted_mean['recency'] = pd.cut(date_submitted_mean['date_submitted'], bins = bins, labels = labels)

In [26]:
 date_submitted_mean['date_submitted'].max()

285.0

**Считаем F-завершаемость**. Смотрим, разброс в количестве экзаменов, которые мог сдать один студент. Видим, что студен за весь период обучение мог попытаться сдать либо один, либо два экзамена.

In [27]:
st_assessment.groupby('id_student',as_index=False).agg({'id_assessment':'count'})['id_assessment'].value_counts()

1    4307
2     326
Name: id_assessment, dtype: int64

Создадим столбец, в котором будет содержаться категориальная переменная, сдал ли экзамен студент или нет.

In [28]:
st_assessment['status'] = st_assessment['score'].apply(lambda x: 'pass' if x >= 40 else 'no_pass')

Подсчитаем долю сданных и не сданных экзаменов. Будем ориентироваться на столбец **pass**, так как его проще интерпретировать. Мы знаем, что студент мог сдавать либо один, либо два экзамена. Соотвественно, 1 ознает, что у студента стопроцентная завершаемость курсов, 0.5 говорит о том, что студент сдал половину от всех экзменов, NaN в нашем случае означает, что студент не сдал ни одного экзамена. Заменим NaN-ы нулями, чтобы затем было удобнее создать столбец с категоиальными переменными.

In [29]:
success_share = st_assessment.groupby('id_student')['status']\
          .value_counts(normalize=True)\
          .to_frame().unstack()\
          .sort_values(by=('status','pass'))\
          .reset_index().droplevel(0, axis=1)

In [30]:
success_share['pass'].value_counts()

1.0    4069
0.5      28
Name: pass, dtype: int64

In [31]:
success_share['pass'].fillna(0, inplace = True)

1 - 100% завершаемость курсов; 2 - 50% завершаемость; 3 - ни сдал ни одного экзамена

In [32]:
success_share['frequency'] = success_share['pass'].apply(lambda x: 1 if x == 1 else (2 if x == 0.5 else 3))
success_share.rename(columns = {'':'id_student'}, inplace = True)

In [33]:
success_share.head()

status,id_student,no_pass,pass,frequency
0,572382,0.5,0.5,2
1,561848,0.5,0.5,2
2,529078,0.5,0.5,2
3,554205,0.5,0.5,2
4,620243,0.5,0.5,2


Посчитаем **M - среднее количество баллов**.

Группируем по уникальному id студента и считаем средний балл по всем экзаменам.

In [34]:
score_mean = st_assessment.groupby('id_student',as_index=False).agg({'score':'mean'})

В первый интервал вошли студенты, которые не сдали экзамен, так как не набрали 40  и более баллов.  В интервале 40 - значение квантиля(75) находятся среднячки, это наиболее обширная группа студентов; нижней границе этого интервала  соотвествует квантиль 0.13. В интервале значений квантилей 0.75 и 1 находятся студенты с самым высокими баллами.

3 - студент не сдал экзамен, 2 - студент-среднячок, 1 - студент большой молодец, набрал много баллов.

In [35]:
score_mean['monetary'] = pd.cut(score_mean['score'], 
                  bins = [0,39,score_mean['score'].quantile(q=0.75),score_mean['score'].max()], 
                  include_lowest = True,
                  labels = [1,2,3])

Добавим получившиеся срезы в новую таблицу.

In [36]:
data_frames = [st_assessment, 
               date_submitted_mean[['recency','id_student']], 
               score_mean[['monetary','id_student']],
               success_share[['frequency','id_student']]]

In [37]:
rgm = reduce(lambda  left,right: pd.merge(left,right,on=['id_student'],
                                            ), data_frames)

|     Переменная    |      Время (R)           |         Завершаемость (F)      |            Кол-во баллов (M)          | 
|       :---        |       :----:             |             :----:             |              ---:                     |
|       1           |       229 - 234          |   Сданы все экзамены           |              0 - 39                   |
|       2           |       235 - 243          |  Сдана половина экзаменов      |              40 - 82                  |
|       3           |       244 - 285          |   Не сдан ни один экзамен      |              83 - 100                 |

Функция для быстрой фильтрации

In [38]:
def rgm(r,f,m, df=rgm):
    return df.query('recency == @r and frequency == @f and monetary == @m')

In [39]:
rgm(1,2,1)

,id_assessment,id_student,date_submitted,is_banked,score,assessment_type,code_module,code_presentation,status,recency,monetary,frequency
19,24290,566185,230,0,36.0,Exam,CCC,2014B,no_pass,1,1,2
20,25354,566185,237,0,40.0,Exam,DDD,2013J,pass,1,1,2
